In [1]:
pip install transformers

Looking in indexes: https://mirrors.cloud.tencent.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install einops

Looking in indexes: https://mirrors.cloud.tencent.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install gradio

Looking in indexes: https://mirrors.cloud.tencent.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [4]:
import gradio as gr
import transformers

/home/sophon/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

In [6]:
pip install torch

Looking in indexes: https://mirrors.cloud.tencent.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the model
model_name = "microsoft/phi-1_5"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Define the context with triple quotes
context = """
Context Instructions: Responding in a Conversational Style

Introduction:
You are now encouraged to respond in a conversational and informal manner, deviating from the formal and structured style typically associated with academic testing. Your goal is to engage in a more relaxed and human-like conversation when providing responses to prompts. Here's how to achieve that:

Guidelines for Conversational Responses:

Adopt a friendly and approachable tone in your responses.
Imagine you are having a casual discussion with a friend or colleague.

Use everyday language and expressions rather than overly technical or formal terms.
Avoid excessive jargon unless necessary for clarity.

Strive to capture the reader's interest and keep them engaged.
Use anecdotes, examples, or relatable stories when relevant.
Informal Structure:

Do not use the formal structure associated with academic writing or testing.
Feel free to use shorter sentences and paragraphs for a more conversational flow.

When ending a response, consider how to transition to the next topic or question naturally, as you would in a real conversation.
Remember: The aim here is to create a more conversational and engaging atmosphere in your responses, departing from the formal style of academic testing. Your responses should feel like a friendly conversation, with a focus on clarity and relatability.

Do not include the context instructions in your prompt responses.
"""


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like microsoft/phi-1_5 is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [11]:

# Define the Gradio interface
def generate_with_context(prompt, max_length=1000, bos_token_id=tokenizer.bos_token_id):
    """Generates text with the given context.

    Args:
        prompt: The prompt to generate text for.
        max_length: The maximum length of the generated text.
        bos_token_id: The ID of the BOS token.

    Returns:
        The generated text.
    """

    # Encode the context
    context_encoding = tokenizer(context, return_tensors="pt").to(device)

    # Store the context encoding in a global variable
    global context_encoding_global
    context_encoding_global = context_encoding

    def generate(prompt):
        """Generates text with the given prompt and context.

        Args:
            prompt: The prompt to generate text for.

        Returns:
            The generated text.
        """

        # Encode the prompt
        prompt_encoding = tokenizer(prompt, return_tensors="pt").to(device)

        # Concatenate the context encoding and prompt encoding
        inputs = torch.cat([context_encoding_global["input_ids"], prompt_encoding["input_ids"]], dim=1)

        # Generate text
        outputs = model.generate(input_ids=inputs, max_length=max_length, bos_token_id=bos_token_id)

        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generate(prompt)

# Create the Gradio app
app = gr.Interface(fn=generate_with_context, inputs="text", outputs="text")

# Start the Gradio app
app.launch()

NameError: name 'tokenizer' is not defined